In [3]:
import numpy as np 
import pandas as pd
import pulp
from itertools import combinations 

---
**Objective:** Maximize profit from producing and shipping the optimal number of products from 4 possible production plants to 7 possible distribution facilities.

Cost Objective = sales - shipping costs - production costs
****
**Constraints:**

Max capacity = total production_plant <= annual capacity_plant

Max demand = total production_distributor < demand_distributor 
****
**Available data:**

The Guardsman_lock file contains three spreadsheets
* Shipping: This sheet contains the cost to ship from each of the plants to each of the distributors
* Distributor: This sheet contains the max demand and sales price for each distributor
* Plants: This sheet contains the cost of producing each unit and the annual capacity for each plant
---




In [4]:
#Load the file into separate dataframes
filepath = '/Volumes/Elements/khartless/Projects/Data/'
file_name = 'guardsman_lock.xlsx'

shipping = pd.read_excel(filepath+file_name, sheet_name= 'Shipping', index_col=[0])
distributor = pd.read_excel(filepath+file_name, sheet_name= 'Distributor', index_col=[0])
plants = pd.read_excel(filepath+file_name, sheet_name= 'Plants', index_col=[0])


In [5]:
shipping

,Tacoma,San Diego,Dallas,Denver,St. Louis,Tampa,Baltimore
Shipping Costs,,,,,,,
Macon,2.50,2.75,1.75,2.00,2.1,1.80,1.65
Louisville,1.85,1.90,1.50,1.60,1.0,1.90,1.85
Detroit,2.30,2.25,1.85,1.25,1.5,2.25,2.00
Phoenix,1.90,0.90,1.60,1.75,2.0,2.50,2.65


In [6]:
distributor

,Tacoma,San Diego,Dallas,Denver,St. Louis,Tampa,Baltimore
Distributor Variables,,,,,,,
Demand,8500.0,14500.00,13500,12600.0,18000.0,15000.00,9000.0
Sales price,42.5,41.75,45,42.9,40.4,41.85,44.2


In [7]:
plants

,Macon,Louisville,Detroit,Phoenix
Plant Variable,,,,
Unit Cost,35.5,37.5,39,36.25
Annual capacity,18000.0,22000.0,25000,20000.00


In [8]:
#determine the number of plants and distributors
n_distributors = len(distributor.columns)
n_plants = len(plants.columns)

n_distributors, n_plants

(7, 4)

In [27]:
#Create the model
model = pulp.LpProblem(name='shipping_planning_guardsman', sense=pulp.LpMaximize)

#Create a matrix of possible production values for each plant and distributor combination
variable_name = [str(i)+str(j) for i in range(1, n_plants+1) for j in range(1, n_distributors+1)]

production_variables = pulp.LpVariable.matrix('P', variable_name,
                                             lowBound=0, cat=pulp.LpContinuous)

#This array of the production variables will be used in calculating the cost function and constraints
allocation = np.array(production_variables).reshape(4,7)                                             

#set constraint for each plant, production at each plant must be less than or equal to max capacity for plant
for index, plant in enumerate(plants.columns): 
    model.addConstraint(pulp.LpConstraint(
        e=pulp.lpSum(allocation[index,:]),
        sense=pulp.LpConstraintLE,
        name='plant_max' + str(plant),
        rhs=plants[plant]['Annual capacity']))

#set demand constraint for each distributor, production allocated to each distributor must be less than or equal to demand
for index, demand in enumerate(distributor.loc['Demand']):
    model.addConstraint(pulp.LpConstraint(
        e=pulp.lpSum(allocation[:,index]),
        sense=pulp.LpConstraintLE,
        name='demand' + str(distributor.columns[index]),
        rhs=demand
    ))

#set objective
total_sales = pulp.lpSum(allocation*distributor.loc['Sales price'].values)
total_shipping_costs = pulp.lpSum(allocation*shipping.values) 
total_unit_costs = pulp.lpSum(allocation[index,:]*cost for index, cost in enumerate(plants.loc['Unit Cost']))                                

objective = total_sales - total_shipping_costs - total_unit_costs

model.setObjective(objective)

model.solve()

#if the model is optimal, print the objective value and the production matrix
if model.status == 1:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')
    print(f'objective: ${model.objective.value():,.0f}')
    output = []
    for i,var in enumerate(model.variables()):
        output.append(var.value())

    print(np.array(output).reshape(4,7))
else:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')

status: 1, Optimal
objective: $333,835
[[    0.     0.     0.     0.     0. 15000.  3000.]
 [ 3000.     0.  7100.     0. 11900.     0.     0.]
 [    0.     0.  6400. 12600.     0.     0.  6000.]
 [ 5500. 14500.     0.     0.     0.     0.     0.]]
